In [1]:
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [2]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    
    in_text = seed_text
    # generate a fixed number of words

    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # pre-pad sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        print('yhat:',yhat)
        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word 
                break
        # append to input
        in_text += ' ' + out_word
    return in_text

In [3]:
# define the model
def define_model(vocab_size, max_length):
    
    model = Sequential()
    model.add(Embedding(vocab_size, 10, input_length=max_length-1))
    model.add(LSTM(50))
    model.add(Dense(vocab_size, activation='softmax'))

    # compile network
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # summarize defined model
    model.summary()
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [4]:
# source text
data = """ Jack and Jill went up the hill\n
To fetch a pail of water\n
Jack fell down and broke his crown\n
And Jill came tumbling after\n """

In [5]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [6]:
encoded

[2,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 2,
 14,
 15,
 1,
 16,
 17,
 18,
 1,
 3,
 19,
 20,
 21]

In [7]:
len(encoded)

25

In [8]:
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 22


In [9]:
# encode 2 words -> 1 word

sequences = list()

for i in range(2, len(encoded)):
    sequence = encoded[i-2:i+1]
    sequences.append(sequence)
    
    
print('Total Sequences: %d' % len(sequences))

Total Sequences: 23


In [10]:
sequences

[[2, 1, 3],
 [1, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9],
 [8, 9, 10],
 [9, 10, 11],
 [10, 11, 12],
 [11, 12, 13],
 [12, 13, 2],
 [13, 2, 14],
 [2, 14, 15],
 [14, 15, 1],
 [15, 1, 16],
 [1, 16, 17],
 [16, 17, 18],
 [17, 18, 1],
 [18, 1, 3],
 [1, 3, 19],
 [3, 19, 20],
 [19, 20, 21]]

In [11]:
# pad sequences

max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

print('Max Sequence Length: %d' % max_length)

Max Sequence Length: 3


In [12]:
# split into input and output elements

sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]


In [13]:
X

array([[ 2,  1],
       [ 1,  3],
       [ 3,  4],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [11, 12],
       [12, 13],
       [13,  2],
       [ 2, 14],
       [14, 15],
       [15,  1],
       [ 1, 16],
       [16, 17],
       [17, 18],
       [18,  1],
       [ 1,  3],
       [ 3, 19],
       [19, 20]])

In [14]:
y

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  2, 14, 15,  1, 16, 17,
       18,  1,  3, 19, 20, 21])

In [15]:
y = to_categorical(y, num_classes=vocab_size)

In [16]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [17]:
# define model
model = define_model(vocab_size, max_length)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 10)             220       
_________________________________________________________________
lstm (LSTM)                  (None, 50)                12200     
_________________________________________________________________
dense (Dense)                (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________


In [18]:
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
1/1 - 0s - loss: 3.0906 - accuracy: 0.0000e+00
Epoch 2/500
1/1 - 0s - loss: 3.0897 - accuracy: 0.0870
Epoch 3/500
1/1 - 0s - loss: 3.0888 - accuracy: 0.0870
Epoch 4/500
1/1 - 0s - loss: 3.0879 - accuracy: 0.0870
Epoch 5/500
1/1 - 0s - loss: 3.0869 - accuracy: 0.0870
Epoch 6/500
1/1 - 0s - loss: 3.0860 - accuracy: 0.1304
Epoch 7/500
1/1 - 0s - loss: 3.0851 - accuracy: 0.1304
Epoch 8/500
1/1 - 0s - loss: 3.0841 - accuracy: 0.1304
Epoch 9/500
1/1 - 0s - loss: 3.0832 - accuracy: 0.1304
Epoch 10/500
1/1 - 0s - loss: 3.0822 - accuracy: 0.1304
Epoch 11/500
1/1 - 0s - loss: 3.0812 - accuracy: 0.1304
Epoch 12/500
1/1 - 0s - loss: 3.0801 - accuracy: 0.1304
Epoch 13/500
1/1 - 0s - loss: 3.0791 - accuracy: 0.1739
Epoch 14/500
1/1 - 0s - loss: 3.0780 - accuracy: 0.1739
Epoch 15/500
1/1 - 0s - loss: 3.0769 - accuracy: 0.1739
Epoch 16/500
1/1 - 0s - loss: 3.0758 - accuracy: 0.1739
Epoch 17/500
1/1 - 0s - loss: 3.0746 - accuracy: 0.1739
Epoch 18/500
1/1 - 0s - loss: 3.0734 - accuracy: 0.17

Epoch 147/500
1/1 - 0s - loss: 1.2311 - accuracy: 0.7826
Epoch 148/500
1/1 - 0s - loss: 1.2069 - accuracy: 0.8261
Epoch 149/500
1/1 - 0s - loss: 1.1828 - accuracy: 0.8696
Epoch 150/500
1/1 - 0s - loss: 1.1589 - accuracy: 0.8696
Epoch 151/500
1/1 - 0s - loss: 1.1353 - accuracy: 0.8696
Epoch 152/500
1/1 - 0s - loss: 1.1119 - accuracy: 0.8696
Epoch 153/500
1/1 - 0s - loss: 1.0888 - accuracy: 0.8696
Epoch 154/500
1/1 - 0s - loss: 1.0660 - accuracy: 0.8696
Epoch 155/500
1/1 - 0s - loss: 1.0434 - accuracy: 0.8696
Epoch 156/500
1/1 - 0s - loss: 1.0212 - accuracy: 0.8696
Epoch 157/500
1/1 - 0s - loss: 0.9992 - accuracy: 0.8696
Epoch 158/500
1/1 - 0s - loss: 0.9776 - accuracy: 0.8696
Epoch 159/500
1/1 - 0s - loss: 0.9563 - accuracy: 0.9130
Epoch 160/500
1/1 - 0s - loss: 0.9354 - accuracy: 0.9130
Epoch 161/500
1/1 - 0s - loss: 0.9148 - accuracy: 0.9130
Epoch 162/500
1/1 - 0s - loss: 0.8946 - accuracy: 0.9130
Epoch 163/500
1/1 - 0s - loss: 0.8748 - accuracy: 0.9130
Epoch 164/500
1/1 - 0s - loss: 

Epoch 291/500
1/1 - 0s - loss: 0.1125 - accuracy: 0.9565
Epoch 292/500
1/1 - 0s - loss: 0.1118 - accuracy: 0.9565
Epoch 293/500
1/1 - 0s - loss: 0.1111 - accuracy: 0.9565
Epoch 294/500
1/1 - 0s - loss: 0.1104 - accuracy: 0.9565
Epoch 295/500
1/1 - 0s - loss: 0.1097 - accuracy: 0.9565
Epoch 296/500
1/1 - 0s - loss: 0.1091 - accuracy: 0.9565
Epoch 297/500
1/1 - 0s - loss: 0.1085 - accuracy: 0.9565
Epoch 298/500
1/1 - 0s - loss: 0.1078 - accuracy: 0.9565
Epoch 299/500
1/1 - 0s - loss: 0.1072 - accuracy: 0.9565
Epoch 300/500
1/1 - 0s - loss: 0.1067 - accuracy: 0.9565
Epoch 301/500
1/1 - 0s - loss: 0.1061 - accuracy: 0.9565
Epoch 302/500
1/1 - 0s - loss: 0.1055 - accuracy: 0.9565
Epoch 303/500
1/1 - 0s - loss: 0.1049 - accuracy: 0.9565
Epoch 304/500
1/1 - 0s - loss: 0.1044 - accuracy: 0.9565
Epoch 305/500
1/1 - 0s - loss: 0.1039 - accuracy: 0.9565
Epoch 306/500
1/1 - 0s - loss: 0.1033 - accuracy: 0.9565
Epoch 307/500
1/1 - 0s - loss: 0.1028 - accuracy: 0.9565
Epoch 308/500
1/1 - 0s - loss: 

Epoch 435/500
1/1 - 0s - loss: 0.0750 - accuracy: 0.9565
Epoch 436/500
1/1 - 0s - loss: 0.0749 - accuracy: 0.9565
Epoch 437/500
1/1 - 0s - loss: 0.0748 - accuracy: 0.9565
Epoch 438/500
1/1 - 0s - loss: 0.0747 - accuracy: 0.9565
Epoch 439/500
1/1 - 0s - loss: 0.0746 - accuracy: 0.9565
Epoch 440/500
1/1 - 0s - loss: 0.0745 - accuracy: 0.9565
Epoch 441/500
1/1 - 0s - loss: 0.0745 - accuracy: 0.9565
Epoch 442/500
1/1 - 0s - loss: 0.0744 - accuracy: 0.9565
Epoch 443/500
1/1 - 0s - loss: 0.0743 - accuracy: 0.9565
Epoch 444/500
1/1 - 0s - loss: 0.0742 - accuracy: 0.9565
Epoch 445/500
1/1 - 0s - loss: 0.0741 - accuracy: 0.9565
Epoch 446/500
1/1 - 0s - loss: 0.0740 - accuracy: 0.9565
Epoch 447/500
1/1 - 0s - loss: 0.0740 - accuracy: 0.9565
Epoch 448/500
1/1 - 0s - loss: 0.0739 - accuracy: 0.9565
Epoch 449/500
1/1 - 0s - loss: 0.0738 - accuracy: 0.9565
Epoch 450/500
1/1 - 0s - loss: 0.0737 - accuracy: 0.9565
Epoch 451/500
1/1 - 0s - loss: 0.0737 - accuracy: 0.9565
Epoch 452/500
1/1 - 0s - loss: 

In [19]:
# evaluate model

In [20]:
""" Jack and Jill went up the hill\n
To fetch a pail of water\n
Jack fell down and broke his crown\n
And Jill came tumbling after\n """

' Jack and Jill went up the hill\n\nTo fetch a pail of water\n\nJack fell down and broke his crown\n\nAnd Jill came tumbling after\n '

In [21]:
print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 1))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
yhat: [3]
Jack and jill


In [22]:
print(generate_seq(model, tokenizer, max_length-1, 'And Jill', 1))

yhat: [19]
And Jill came


In [23]:
print(generate_seq(model, tokenizer, max_length-1, 'came tumbling', 1))

yhat: [21]
came tumbling after


In [24]:
print(generate_seq(model, tokenizer, max_length-1, 'tumbling after', 1))

yhat: [21]
tumbling after after


In [25]:
print(generate_seq(model, tokenizer, max_length-1, 'Jill went', 5))

yhat: [5]
yhat: [6]
yhat: [7]
yhat: [8]
yhat: [9]
Jill went up the hill to fetch
